River Kelly

Kyler Gappa

CSCI-347

Project 03: Dimensinality Reduction and Clustering

This project may be completed individually or with group of up to size three. Turn in the code
and written responses in both Brightspace and Gradescope.

Choose a data set that you are interested in from the UCI Machine Learning Repository that has
at least five numerical attributes, and that you believe may contain clusters. Only use the numerical
attributes for this project. Note: if you are planning to complete the extra credit portion of this
project, you will need to use a data set that has class labels (ground truth cluster labels), i.e., a
classification data set, in order to compute the accuracy of the clustering. If you would like to use
a data set from a different source, please discuss this with me.

# Part 1: Think about the data

  This data is interesting because it takes a unique approach to language analysis. An understanding of writing could help us recognise and decipher currently unreadable text. There are six numerical and zero categorical attributes. The repository says that there are no missing attributes in the dataset so no extra techniques will be used unless missing values are found. we expect that there will be a few clusters due to a few different reasons. There is a chance for clusters for each number (i.e. all 4's will form a cluster) and a chance for clusters of every number from people that write similarly. If clusters exist then we can use the range of values that fit within the cluster to help identify similar letters in the future. We expect for there to be four to ten clusters as there a few numbers that may have similar shapes or may have consistent shapes that are mathmatically similar. We expect there to be some clusters that are differing sizes as there are some shapes that more numbers fall into and that should result in a larger cluster (i.e. 6, 8, 9 vs 1, 7).

# Part 2: Write Python code for clustering

Write the following functions in Python. You may use scikit-learn or other packages to check
the correctness of your implementation, but you may not use any existing clustering algorithm
implementation in your code.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

## 1. (10 points) *k*-means Clustering Algorithm

A function that implements the $k$-means clustering
algorithm. The function should take a data matrix, a number of clusters $k$,
and a convergence parameter $\epsilon$, as input, and return the
representatives (means) as well as the clusters found using $k$-means. If
the distance is the same between a point and more than one representative
(mean), then assign the point to the mean corresponding to the cluster with
the lowest index.

In [ ]:
def kMeanClustering(D, k_cluster: int, convergence = 0.0):
    # var: points_dict
    # --------------------------------------------------------------------------
    # disctionary of points (x, y) from the data 'D'
    points_dict = dict()

    # populate points_dict
    for index, point_data in enumerate(D):
        points_dict[index] = (point_data[0], point_data[1])
    # sort the points_dict
    points_dict = {k: v for k, v in sorted(points_dict.items(), key=lambda item: item[1][1])}
    

    clusters = dict() # dictionary of clusters
    
    # init_points_indices
    # -------------------
    # list to help ensure that each cluster has a unique randomly selected point
    init_points_indices = list() 
    # init clusters with random points
    for i in range(k_cluster):
        init_point_index = random.randint(0, D.shape[0]-1)
        while init_point_index in init_points_indices:
            init_point_index = random.randint(0, D.shape[0]-1)
        init_points_indices.append(init_point_index)
        init_point = points_dict[init_point_index]
        # make cluster item
        clusters[i] = {
            'mean': (init_point[0], init_point[1]),
            'points_indices': list()
        }

    # return
    cluster_iters = list()
    count_iter = 0
    while True:
        count_iter += 1
        if count_iter > 10000:
            break
        # add points to clusters
        for (point_index, point) in points_dict.items():
            cluster_to_point_distances_dict = dict()
            for (cluster_index, cluster) in clusters.items():
                distance_x = cluster['mean'][0] - point[0]
                distance_y = cluster['mean'][1] - point[1]
                distance = ( (distance_x ** 2) + (distance_y ** 2) ) ** (1/2)
                cluster_to_point_distances_dict[cluster_index] = distance
            # sort
            cluster_to_point_distances_dict = {k: v for k, v in sorted(cluster_to_point_distances_dict.items(), key=lambda item: item[1])}
            closest_cluster_index = list(cluster_to_point_distances_dict.keys())[0]

            # add point to closest cluser
            clusters[closest_cluster_index]['points_indices'].append(point_index)
            # update cluster mean
            x_sum = y_sum = 0
            for cluster_point_index in clusters[closest_cluster_index]['points_indices']:
                cluster_point = points_dict[cluster_point_index]
                x_sum += cluster_point[0]
                y_sum += cluster_point[1]
            num_cluster_points = len(clusters[closest_cluster_index]['points_indices'])
            x_mean = x_sum / num_cluster_points
            y_mean = y_sum / num_cluster_points
            clusters[closest_cluster_index]['mean'] = (x_mean, y_mean)


        total_distance_sum = 0
        for (cluster_index, cluster) in clusters.items():
            distance = 0
            for point_index in cluster['points_indices']:
                point = points_dict[point_index]
                distance += (((point[0]-cluster['mean'][0])**2)+((point[1]-cluster['mean'][1])**2))**(1/2)
            clusters[cluster_index]['dist_sum'] = distance
            total_distance_sum += distance

        cluster_iters.append({
            'clusters': clusters,
            'total_distance': total_distance_sum
        })

        if len(cluster_iters) > 1:
            last_cluster_dist = cluster_iters[-2]['total_distance']
            dist_diff = abs(total_distance_sum - last_cluster_dist)
            if dist_diff <= convergence:
                break

        # remove cluster points for next iteration
        new_clusters = dict()
        for (i, cluster) in clusters.items():
            new_clusters[i] = {
                'mean': (cluster['mean'][0], cluster['mean'][1]),
                'points_indices': list()
            }
        clusters = new_clusters

    clusters = cluster_iters[-1]['clusters']
    # print(cluster_iters[-1]['total_distance'])


    pred_labels = [0] * D.shape[0]
    for cluster_index in clusters:
        cluster = clusters[cluster_index]
        for point in cluster['points_indices']:
            pred_labels[point] = cluster_index
    
    centers = np.ndarray(shape=(k_cluster, 2))
    for i, cluster in enumerate(clusters.values()):
        centers[i] = cluster['mean']

    plt.scatter(D[:,0], D[:,1], c=pred_labels)
    plt.scatter(centers[:,0], centers[:,1], c='red')

In [ ]:
from sklearn.datasets import make_blobs
D, labels = make_blobs(n_samples=100, centers=3, cluster_std=.3, random_state=0)

In [ ]:
plt.scatter(D[:,0], D[:,1], c=labels)
plt.xlabel("X1")
plt.ylabel("X2")
plt.title('2d clusters')

In [ ]:
kMeanClustering(D, 3, 0.5)

## 2. (10 points) DBSCAN Clustering Algorithm

A function that implements the DBSCAN clustering
algorithm. The function should take a data matrix and the parameters
*minpts* and $\epsilon$, as input, and return the clusters found using
DBSCAN, and for each data point a label of core, border, or noise point.

## 3. (Extra Credit - 5 points) Precision of Clustering

A function that computes the precision of a
clustering. The function should take a list of true cluster labels and a
list of the cluster labels returned by some clustering algorithm, and return
the precision of the clustering.

# Part 3: Analyze your data

## 1. (4 points)

Use sklearn's PCA implementation to linearly transform the
data to two dimensions. Create a scatter plot of the data, with the $x$-axis
corresponding to coordinates of the data along the first principal
component, and the $y$-axis corresponding to coordinates of the data along
the second principal component. Does it look like there are clusters in
these two dimensions? If so, how many would you say there are?

## 2. (3 points)

Use sklearn's PCA implementation to linearly transform the
data, without specifying the number of components to use. Create a plot with
$r$, the number of components (i.e., dimensionality), on the $x$-axis, and
$f(r)$, the fraction of total variance captured in the first $r$ principal
components, on the $y$-axis. Based on this plot, choose a number of
principal components to reduce the dimensionality of the data. Report how
many principal components will be used as well as the faction of total
variance captured using this many components.

## 3. (5 points)

For both the original and the reduced-dimensionality data
obtained using PCA in question 3, do the
following: Experiment with a range of values for the number of clusters,
$k$, that you pass as input to the $k$-means function, to find clusters in
the chosen data set. Use at least 5 different values of $k$. For each value
of $k$, report the value of the objective function for that choice of $k$.

## 4. (5 points)

For both the original and the reduced-dimensionality data
obtained using PCA in question 3, do the following:
Experiment with a range of values for the *minpts* and $\epsilon$ input
parameters to the DBSCAN function to find clusters in the chosen data set.
First, keep $\epsilon$ fixed and try out a range of different values for
*minpts*. Then keep *minpts* fixed, and try a range of values for
$\epsilon$. Use at least 5 values of $\epsilon$ and at least 5 values of
*minpts*. Report the number of clusters found for each (*minpts*,
$\epsilon$) pair tested.

## 5. (Extra credit - 3 points)

Create a plot of clustering precision for
each value of $k$ used in question 3, each value of
$\epsilon$ used in question 3, and each value of
*minpts* used in question 3, for both the original
and reduced-dimensionality data.

# Tips and Acknowledgements

Make sure to submit your answer as a PDF on Gradscope and Brightspace. Make sure
to show your work. Include any code snippets you used to generate an answer,
using comments in the code to clearly indicate which problem corresponds to
which code.

**Acknowledgements**: Project adapted from assignments of Veronika Strnadova-Neeley.